In [1]:
%load_ext autoreload
%autoreload 2

Using Feature Enginering

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import RepeatedStratifiedKFold


In [2]:
df = pd.read_csv('combined_features.csv',index_col=0)
df.sort_index(inplace=True)

In [3]:
df

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread,forward_return,output
time,,,,,,,,,,,
2020-11-17 17:00:00,0.949281,0.911684,0.908723,0.910738,1.028341,0.291573,0.913092,1.458798,8.522631,-0.147019,-1
2020-11-18 10:00:00,0.949618,0.912712,0.910101,0.912506,1.154188,0.337144,1.264965,2.788685,7.923738,-0.195590,-1
2020-11-18 11:00:00,0.949919,0.914357,0.912081,0.915681,1.443286,0.416619,1.582164,2.204730,7.326875,-0.288873,-1
2020-11-18 12:00:00,0.950429,0.917156,0.914294,0.919506,1.567864,0.517426,2.864586,11.746214,6.784229,-0.396706,-1
2020-11-18 13:00:00,0.950870,0.920886,0.916310,0.922577,1.556466,0.605077,3.047215,13.277380,6.601599,-0.430397,-1
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28 13:00:00,0.948541,0.837141,0.838733,0.836955,0.914318,-0.267499,-1.990000,0.492807,14.850000,0.071380,1
2022-10-28 14:00:00,0.948539,0.836728,0.838468,0.837909,0.968452,-0.254640,0.225000,9.559848,14.535000,0.054696,1
2022-10-28 15:00:00,0.948534,0.837559,0.838659,0.838728,0.973852,-0.245484,-0.322900,15.006942,14.582900,0.050257,1


In [4]:
#generate label for training
import numpy as np
def gen_labels(df,t,threshold):
    # Calculate % return on spread t hours later
    df['forward_return'] = df['spread'].diff(periods=t)/df['spread']
    
    #If the return is more than x%, we should have bought, and hence the label is (1)
    #If return is less than x%, we should have sold, and hence label is (-1)
    #If in between, do nothing (0)
    df['output'] = np.select([df['forward_return'] > threshold ,df['forward_return'] < -threshold],[1,-1])
    return df

df = gen_labels(df,t=24,threshold=0.05)
df

,vwap,SMA(5),SMA(10),12dayEWM,rsi,MACD,mom,mfi,spread,forward_return,output
time,,,,,,,,,,,
2020-11-17 17:00:00,0.949281,0.911684,0.908723,0.910738,1.028341,0.291573,0.913092,1.458798,8.522631,NaN,0
2020-11-18 10:00:00,0.949618,0.912712,0.910101,0.912506,1.154188,0.337144,1.264965,2.788685,7.923738,NaN,0
2020-11-18 11:00:00,0.949919,0.914357,0.912081,0.915681,1.443286,0.416619,1.582164,2.204730,7.326875,NaN,0
2020-11-18 12:00:00,0.950429,0.917156,0.914294,0.919506,1.567864,0.517426,2.864586,11.746214,6.784229,NaN,0
2020-11-18 13:00:00,0.950870,0.920886,0.916310,0.922577,1.556466,0.605077,3.047215,13.277380,6.601599,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28 13:00:00,0.948541,0.837141,0.838733,0.836955,0.914318,-0.267499,-1.990000,0.492807,14.850000,0.071380,1
2022-10-28 14:00:00,0.948539,0.836728,0.838468,0.837909,0.968452,-0.254640,0.225000,9.559848,14.535000,0.054696,1
2022-10-28 15:00:00,0.948534,0.837559,0.838659,0.838728,0.973852,-0.245484,-0.322900,15.006942,14.582900,0.050257,1


In [5]:
#train and test data
split = round(0.8*len(df))
train, test = df[:split],df[split:]
x_train = train[['vwap','SMA(5)','SMA(10)','12dayEWM','rsi','MACD','mom','mfi','spread']]
y_train = train[['output']]

x_test = test[['vwap','SMA(5)','SMA(10)','12dayEWM','rsi','MACD','mom','mfi','spread']]
y_test = test[['output']]

In [6]:

# param_grid={'penalty':['l1','l2'], 'C':[1e-3, 1e-2, 1e-1, 1, 10]}

# # Create a based model
# lr = LogisticRegression()
# # Instantiate the grid search model
# model = GridSearchCV(estimator=LogisticRegression(solver="liblinear"), param_grid=param_grid) #helps to test different kinds of hype params
# model.fit(x_train,y_train)
# y_pred = model.predict(x_test)

Testing all hyperparams and solvers

In [12]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l1','l2', 'none', 'elasticnet']
c_values = [1e-3, 1e-2, 1e-1, 1, 10]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = GridSearchCV(estimator=LogisticRegression(), param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = model.fit(x_train, y_train.values.ravel())
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks

Best: 0.708832 using {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'lbfgs'}
0.460144 (0.030674) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'sag'}
0.441376 (0.024413) with: {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
0.519505 (0.028662) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'newton-cg'}
0.519505 (0.028662) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
0.523405 (0.029781) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'liblinear'}
0.519505 (0.028662) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'sag'}
0.520876 (0.027983) with: {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}
0.708832 (0.023644) with: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
0.700612 (0.022489) with: {'C': 0.001, 'penalty': 'none', 'solver': 'lbfgs'}
0.0000

In [13]:
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_pred, y_test))
print(model.best_params_)

              precision    recall  f1-score   support

          -1       0.96      0.40      0.57       178
           0       0.57      0.95      0.71       386
           1       0.76      0.22      0.35       227

    accuracy                           0.62       791
   macro avg       0.76      0.53      0.54       791
weighted avg       0.71      0.62      0.57       791

0.6194690265486725
{'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}


In [14]:
joblib.dump(model, 'logreg.pkl')

['logreg.pkl']